In [2]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib
import matplotlib.pyplot as plt
import math
import pickle
import os
import cv2
from PIL import Image
import random


In [10]:
#load images
images = {}
for alphabet in ["A","B", "C", "D", "E", "F", "G", "H", "I", "J"]:
    images[alphabet] = []
    for filename in os.listdir(r"notMNIST_small/" + alphabet):
        img = cv2.imread("notMNIST_small/"+ alphabet + "/" + filename, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(alphabet, filename)
            continue
        img = img.reshape(28*28,)
        images[alphabet].append([img])


A .DS_Store
F .DS_Store


In [5]:
# split train, valid and test sets
train_set = []
valid_set = []
test_set = []

train_target = []
valid_target = []
test_target = []

target_set = {"A": [1,0,0,0,0,0,0,0,0,0],
              "B": [0,1,0,0,0,0,0,0,0,0],
              "C": [0,0,1,0,0,0,0,0,0,0],
              "D": [0,0,0,1,0,0,0,0,0,0],
              "E": [0,0,0,0,1,0,0,0,0,0],
              "F": [0,0,0,0,0,1,0,0,0,0],
              "G": [0,0,0,0,0,0,1,0,0,0],
              "H": [0,0,0,0,0,0,0,1,0,0],
              "I": [0,0,0,0,0,0,0,0,1,0],
              "J": [0,0,0,0,0,0,0,0,0,1]}

target_set = {"A": [0],
              "B": [1],
              "C": [2],
              "D": [3],
              "E": [4],
              "F": [5],
              "G": [6],
              "H": [7],
              "I": [8],
              "J": [9]}

for alphabet in ["A","B", "C", "D", "E", "F", "G", "H", "I", "J"]:
    random.shuffle(images[alphabet])
    train_set.extend(images[alphabet][:1500])
    for i in range(1500): train_target.append(target_set[alphabet])

    valid_set.extend(images[alphabet][1500:1600])
    for i in range(100): valid_target.append(target_set[alphabet])

    test_set.extend(images[alphabet][1600:])
    for i in range(len(images[alphabet][1600:])): test_target.append(target_set[alphabet])


    
train_set = torch.from_numpy(np.array(train_set)).float()
valid_set = torch.from_numpy(np.array(valid_set)).float()
test_set = torch.from_numpy(np.array(test_set)).float()

train_target = torch.from_numpy(np.array(train_target)).long()
valid_target = torch.from_numpy(np.array(valid_target)).long()
test_target = torch.from_numpy(np.array(test_target)).long()




In [60]:
# model
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(28*28, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return F.softmax(x)


In [61]:
# model
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(28*28, 100500)
        self.fc2 = nn.Linear(100500, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return F.softmax(x)


In [4]:
learning_rate = 10

network = Net2()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate)



NameError: name 'Net2' is not defined

In [65]:
def train(epoch):
    learning_rate = 1

    network = nn.Sequential(nn.Linear(28*28,1000), nn.ReLU(), nn.Linear(1000,10),nn.Softmax())
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate)

    train_losses = []
    valid_losses = []
    network.train()

    # random_order = [i for i in range(len(train_set))]
    # random.shuffle(random_order)
    # need to shuffle train data
    for i in range(epoch):
        data = train_set
        target = train_target

        # print(network[0].weight)
        output = network(data)
        loss = criterion(output.transpose(1,2), target)
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print(network[0].weight)
        # print(loss.data.item())

        if i % 1 == 0:
            print('Train Loss: {:.6f}'.format(loss.item()))
        train_losses.append(loss.item())
    




In [82]:
train_pred = network(train_set)
print(train_pred.size())

print(train_target[0])

train_index = [np.argmax(x[0].detach().numpy()) for x in train_pred]
acc_num = 0
for i in range(len(train_index)):
    if train_index[i] == train_target[i][0]:
        acc_num += 1
print(acc_num) 
    

/Users/jimmyzheng/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


torch.Size([15000, 1, 10])
tensor([0])
1182


In [80]:
valid_pred = network(valid_set)
print(valid_pred.size())

print(valid_target[0])

valid_index = [np.argmax(x[0].detach().numpy()) for x in valid_pred]
acc_num = 0
for i in range(len(valid_index)):
    if valid_index[i] == valid_target[i][0]:
        acc_num += 1
print(acc_num)
    

torch.Size([1000, 1, 10])
tensor([0])
78


/Users/jimmyzheng/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


In [67]:
a = np.array([2,5,4,6,3])
print(np.argmax(a))

3


In [66]:
train(30)

Train Loss: 2.302706
Train Loss: 2.302709
Train Loss: 2.302703
Train Loss: 2.302705
Train Loss: 2.302702
Train Loss: 2.302701
Train Loss: 2.302698
Train Loss: 2.302638
Train Loss: 2.302635
Train Loss: 2.302637
Train Loss: 2.302609
Train Loss: 2.302571
Train Loss: 2.302572
Train Loss: 2.302571
Train Loss: 2.302568
Train Loss: 2.302570
Train Loss: 2.302571
Train Loss: 2.302570
Train Loss: 2.302571
Train Loss: 2.302568
Train Loss: 2.302570
Train Loss: 2.302570
Train Loss: 2.302568
Train Loss: 2.302570
Train Loss: 2.302572
Train Loss: 2.302570
Train Loss: 2.302571
Train Loss: 2.302570
Train Loss: 2.302570
Train Loss: 2.302570


In [5]:
criterion = nn.CrossEntropyLoss()
a = torch.Tensor([[1,2,3]])
b = torch.Tensor([1]).long()
print(a.size())
print(b.size())
ll = criterion(a,b)
print(ll)

c = F.softmax(a)
print(c)

torch.Size([1, 3])
torch.Size([1])
tensor(1.4076)
tensor([[0.0900, 0.2447, 0.6652]])


/Users/jimmyzheng/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


In [34]:
print(network.fc1.weight)

Parameter containing:
tensor([[-0.0080,  0.0253, -0.0162,  ..., -0.0201,  0.0044,  0.0328],
        [-0.0244, -0.0154, -0.0177,  ..., -0.0339, -0.0070, -0.0126],
        [-0.0033, -0.0236, -0.0048,  ..., -0.0210,  0.0210, -0.0044],
        ...,
        [-0.0059, -0.0196, -0.0236,  ..., -0.0278,  0.0015,  0.0063],
        [ 0.0071,  0.0212, -0.0192,  ..., -0.0005,  0.0301, -0.0225],
        [-0.0050, -0.0196,  0.0229,  ..., -0.0270, -0.0003, -0.0024]],
       requires_grad=True)


In [92]:

a = []
for i in range(10):
    a.append(b)
a = np.array(a)
print(b.shape)
print(a.shape)

(2, 2)
(10, 2, 2)


In [34]:
a = np.random.rand(1,3,2)
print(a)
a = a.transpose((0,2,1))
print(a)

xx = torch.Tensor([[[0.2,0.3,0.5], [0.4,0.5,0.1]]])
print(xx.size())
print(xx)

xx = xx.transpose(1,2)
print(xx)



[[[0.96775459 0.86893727]
  [0.59530076 0.15473133]
  [0.70550377 0.19061929]]]
[[[0.96775459 0.59530076 0.70550377]
  [0.86893727 0.15473133 0.19061929]]]
torch.Size([1, 2, 3])
tensor([[[0.2000, 0.3000, 0.5000],
         [0.4000, 0.5000, 0.1000]]])
tensor([[[0.2000, 0.4000],
         [0.3000, 0.5000],
         [0.5000, 0.1000]]])


In [98]:
filenames = ""
imgs = cv2.imread("notMNIST_small/"+ "J" + "/" + filenames, cv2.IMREAD_GRAYSCALE)
print(imgs.shape)
a = []
for i in range(1000):
    a.append(imgs)
print(np.array(a).shape)

(28, 28)
(1000, 28, 28)


In [10]:
import torch
import torch.nn as nn

# model
class Models(nn.Module):
    def __init__(self):
        super(Models, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(3, 1000)
        self.fc2 = nn.Linear(1000, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return F.softmax(x)



x_data = torch.Tensor([[0.2,0.3,0.5], [0.4,0.5,0.1]])
y_data = torch.Tensor([0, 1]).long()
model = nn.Sequential(nn.Linear(3,10), nn.ReLU(), nn.Linear(10,2),nn.Softmax())

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1 )

for epoch in range(10) :
    y_pred = model(x_data)
    loss = criterion( y_pred, y_data )
    print(loss)
    print( epoch, loss.data.item() )
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(model[0].weight)
print(y_pred.size())
print(y_data.size())

tensor(0.6800, grad_fn=<NllLossBackward>)
0 0.6799924373626709
Parameter containing:
tensor([[-0.1150,  0.3878, -0.2248],
        [ 0.0953,  0.2315, -0.0741],
        [-0.4820,  0.5382, -0.0987],
        [-0.1808, -0.1805,  0.4229],
        [ 0.1607, -0.1644, -0.1187],
        [-0.5244,  0.5741, -0.0474],
        [ 0.1447,  0.4507,  0.3475],
        [-0.5194,  0.0140,  0.2324],
        [-0.3426, -0.3281,  0.1879],
        [ 0.4064, -0.2274, -0.4419]], requires_grad=True)
tensor(0.6796, grad_fn=<NllLossBackward>)
1 0.6795734167098999
Parameter containing:
tensor([[-0.1148,  0.3880, -0.2252],
        [ 0.0953,  0.2315, -0.0741],
        [-0.4820,  0.5382, -0.0987],
        [-0.1814, -0.1811,  0.4243],
        [ 0.1605, -0.1646, -0.1183],
        [-0.5245,  0.5740, -0.0471],
        [ 0.1447,  0.4507,  0.3475],
        [-0.5194,  0.0140,  0.2324],
        [-0.3426, -0.3281,  0.1879],
        [ 0.4068, -0.2270, -0.4428]], requires_grad=True)
tensor(0.6792, grad_fn=<NllLossBackward>)
2 0.67

/Users/jimmyzheng/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [11]:
print('Number {} : Train Loss: {:.6f}'.format(1,2))

Number 1 : Train Loss: 2.000000


In [12]:
a = "111"
b = "2222"

print(f"Epoch {a}, Training loss: {b}")

Epoch 111, Training loss: 2222
